In [146]:
d_k = torch.tensor([1.])
a_j =  torch.tensor([i.y for i  in model.hidden_layer] + [1])
print(a_j, "a_j")
print(d_k, "d_k")
update = a_j * d_k.view(-1, 1,1)
print("update", update)

output_layer = torch.stack(list(model.output_layer.parameters()))
print("output_layer", output_layer)

updated_weights = output_layer - update
print("update", updated_weights)

with torch.no_grad():
    for param, new_weight in zip(model.output_layer.parameters(), updated_weights):
        param.copy_(new_weight)


print(torch.stack(list(model.output_layer.parameters())))  # [[0., 0., 0.]] # 1,3

tensor([0.7059, 0.6601, 1.0000]) a_j
tensor([1.]) d_k
update tensor([[[0.7059, 0.6601, 1.0000]]])
output_layer tensor([[[ 0.7457,  0.3547, -1.5159]]])
update tensor([[[ 0.0398, -0.3054, -2.5159]]])
tensor([[[ 0.0398, -0.3054, -2.5159]]])


In [130]:
output_layer = torch.tensor([[[-0.0813, -0.1582,  0.5678]], [[-0.0813, -0.1582,  0.5678]]])
d_k = torch.tensor([1.,2.])
print("d_k review", d_k.view(-1, 1,1))
sum_d_j = output_layer * d_k.view(-1, 1,1)
print(sum_d_j)
sum_d_j = torch.sum(sum_d_j, dim=0)
print(sum_d_j)
print(sum_d_j[:, :-1][0])  # ignore the third value as this is the bias.
d_j = [torch.tensor([0.2134]), torch.tensor([0.2496])]
print(torch.cat(d_j))
d_j = torch.cat(d_j) * sum_d_j[:, :-1][0]
print(d_j)

d_k review tensor([[[1.]],

        [[2.]]])
tensor([[[-0.0813, -0.1582,  0.5678]],

        [[-0.1626, -0.3164,  1.1356]]])
tensor([[-0.2439, -0.4746,  1.7034]])
tensor([-0.2439, -0.4746])
tensor([0.2134, 0.2496])
tensor([-0.0520, -0.1185])


In [455]:
# weight update ij
lr = 0.1
x_test = torch.tensor([0., 0., 1.]).reshape(1,-1)
d_j = torch.tensor([-0.0520, -0.1185]).reshape(1,-1)

updated_weights = torch.matmul(d_j.T, x_test) * lr
hl_weights = torch.stack(list(model.hidden_layer.parameters()))
print(hl_weights, "before hl_weights")
updated_weights = updated_weights.reshape(updated_weights.shape[0], 1,updated_weights.shape[1])
print(updated_weights.shape, hl_weights.shape)
# hidden_layer = torch.tensor([[[ 0.5429, -0.5430,  1.3089]], [[-0.1468, -0.7791, -0.5856]]])
hl_weights = hl_weights - updated_weights
print(hl_weights, "after hl_weights")

with torch.no_grad():
    for param, new_weight in zip(model.hidden_layer.parameters(), hl_weights):
        param.copy_(new_weight)
        

# print(torch.stack(list(model.hidden_layer.parameters())))  # [[0., 0., 0.]] # 1,3

tensor([[[ 0.5781, -0.2402, -0.2009]],

        [[-0.4334, -2.2371, -1.2284]]]) before hl_weights
torch.Size([2, 1, 3]) torch.Size([2, 1, 3])
tensor([[[ 0.5781, -0.2402, -0.1957]],

        [[-0.4334, -2.2371, -1.2166]]]) after hl_weights


In [22]:
class NeuralNetwork(nn.Module):
  def __init__(self, num_inputs, num_neurons, num_outputs):
    super().__init__()    
    self.hidden_layer = nn.ModuleList([LogisticRegression(num_inputs) for i in range(num_neurons)])
    self.output_layer = nn.ModuleList([LogisticRegression(num_neurons+1) for i in range(num_outputs)]) # +1 for the bias

  def forward(self, x: torch.tensor) -> torch.tensor:
    y_hidden = torch.ones(len(self.hidden_layer) + 1) # +1 for the bias
    y = torch.ones(len(self.output_layer))  
    #print(f"y hidden: ", y_hidden)
    ### your code here
    y_hidden = torch.tensor([layer(x) for layer in self.hidden_layer] + [1])
    # for i, l in enumerate(self.hidden_layer):
    #     y_hidden[i] = l(x)

    #print(f"y hidden: ", y_hidden)
    for i, l in enumerate(self.output_layer):
        y[i] = l(y_hidden)
    #print(f"y: {y}")
    return y

  def backward_pass(self, x: torch.tensor, y: torch.tensor, lr: float) -> None:
    ### your code here

    #print("length of output layers", len(self.output_layer))
    y_pred = self.forward(x)
    d_outputs = torch.zero(len(self.output_layer))
    d_hidden = torch.ones(len(self.hidden_layer)+1)
    
    y_hidden = torch.tensor([layer(x) for layer in self.hidden_layer] + [1])
    y_out = torch.tensor([layer(y_hidden) for layer in self.output_layer] + [1])

    # error with respect to j.
    # Equivalent to length of hidden neurons
    (y_pred - y) 

    
    for index in range(len(self.output_layer)):
    # To do the backward, we need to understand the partial derivatives and the chain rule, for it to completely make sense.
         d_outputs[index] = ((y_pred[index]-y[index]) * (y_pred[index]*(1-y_pred[index])))


    # sum [[neuron1, neuron2, neuron3] torch.tensor(elem.weight) * d_hidden[index] for index, elem in enumerate(self.output_layer)]
    hidden_sum = []
    for index, elem in enumerate(self.output_layer):
      hidden_sum.append( elem.weight * d_hidden[index])
    hidden_sum = torch.sum(hidden_sum[0], dim=0)
      
    for index in range(len(d_hidden)):
        # sum of all k outputs that specific nodex j goes to. Times the weight value j to k and the output k.
        # once we have done this, the model will be able to achieve better accuracy. 
        d_hidden[index] = y_hidden[index] * (1-y_hidden[index]) * hidden_sum[index]


    # Update weights of output layer. (dimension num_outputs , hidden+1)
    for output_index in range(len(self.output_layer)):
        for index, weight in enumerate(self.output_layer[output_index].weight):
            # print(f"New weight: { lr * d_outputs[output_index] * weight }")
            # print(lr, d_outputs[output_index], weight)
            self.output_layer[output_index].weight[index] =  self.output_layer[output_index].weight[index] - (lr * d_outputs[output_index] * y_out[output_index])


    hidden_weights = torch.stack(list(self.hidden_layer.parameters()))
    hidden_weights = [inner[0].tolist() for inner in hidden_weights]
    # print("self.hidden weights", hidden_weights)
    # [[0.000819722015876323, -0.17005424201488495, -1.459598422050476], [-2.2977514266967773, 0.44948890805244446, 0.5397163033485413]]
    # 2,3 weights
    for i in range(len(list(self.hidden_layer))):
        #print(f"hidden layer {i}")
        #print("weight of hidden layer", self.hidden_layer[i].weight)
        #print("calucalted: ", torch.tensor(self.hidden_layer[i].weight - (lr*x*d_hidden)) )
        self.hidden_layer[i].weight = torch.nn.Parameter(self.hidden_layer[i].weight - (lr*x*d_hidden))
            
    # wjk = delta k * output of j.
    # torch.stack(list(self.output_layer[0]))
     # # [[0., 0., 0.]] # 1,1,3
    #for output_index in range(len(list(model.output_layer))):
        
        
    # print(f"d_outputs", d_outputs)
    # print("output layer parameters", self.output_layer[0].weight)  
    return
